In [1]:
# import block
import json
from time import sleep
import requests
import lxml
from bs4 import BeautifulSoup
from selenium import webdriver
import sqlite3 as lite
import pandas as pd
from pymongo import MongoClient

# Vacation Homework

## Q1. Python Basic, Web Scraping, RDBMS, SQL

### Q1-1

`Ancient Egyptian multiplication`(one of two multiplication methods used by scribes, was a systematic method for multiplying two numbers that does not require the multiplication table, only the ability to multiply and divide by 2, and to add.) 을 Python 으로 구현하세요.

example output)

```
enter two numbers: 24 15
  24      15 struck
  12      30 struck
   6      60 struck
   3     120 keep
   1     240 keep
The result is 360
```

In [2]:
# Q1-1 solution (190127)

# input 2-numbers with space, assign each num to a and b
while True:
    user_input = input("Type two numbers with space:")
    # 스페이스로 구분되는 값이 2개가 아니면 다시 입력하게 한다.
    if len(user_input.split())!=2:
        print("input >>2<< integer numbers with space!")
        continue
    # 2개의 값이 입력되었다면 각 값이 integer인지 확인하고
    # integer가 아닌 값이 입력되어 있다면 오류메시지를 출력하고 다시 입력하게 한다.
    # 2개의 값이 모두 integer라면 a, b 변수에 각각 할당한다.
    try:
        [a, b] = [int(x) for x in user_input.split()]
        break
    except:
        print("Input 2 >>integer<< numbers with space!")

# execute Ancient Egyptian multiplication
i = 0
b_origin = b
valid_exponential_order = []
while True:
    if a%2 == 1:
        valid_exponential_order.append(i)
        status = "keep"
    else:
        status = "struck"
    print("{a}{tab}{b}{tab}{status}".format(a=a, tab='\t', b=b, status=status))
    if a == 1:
        break
    a //= 2
    b *= 2
    i += 1

print("The result is {}.".format(b_origin*sum([2**x for x in valid_exponential_order])))

Type two numbers with space: 24 15


24	15	struck
12	30	struck
6	60	struck
3	120	keep
1	240	keep
The result is 360.


In [2]:
# Q1-1

Type two numbers with space: 24 15
24      15      struck
12      30      struck
6       60      struck
3       120     keep
1       240     keep
The result is 360.


360

### Q1-2

Twitter의 `#asiancup` 에 대한 트윗 검색결과(https://twitter.com/search?q=%23asiancup)를 sqlite 를 활용한 database에 적재하세요.
(10회 스크롤 했을 때의 트윗 본문 전체(이미지 제외))

`Requirements: selenium, sqlite3`

- **Caution** : https://twitter.com/robots.txt 의 지침을 따르세요.

```
# Every bot that might possibly read and respect this file.
User-agent: *
Allow: /*?lang=
Allow: /hashtag/*?src=
Allow: /search?q=%23
Disallow: /search/realtime
Disallow: /search/users
Disallow: /search/*/grid

Disallow: /*?
Disallow: /*/followers
Disallow: /*/following

Disallow: /account/not_my_account
Disallow: /account/deactivated
Disallow: /settings/deactivated

Disallow: /oauth
Disallow: /1/oauth

Disallow: /i/streams
Disallow: /i/hello

# Wait 1 second between successive requests. See ONBOARD-2698 for details.
Crawl-delay: 1
```

In [33]:
#Q1-2

[<selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", element="0.8444793454357549-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", element="0.8444793454357549-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", element="0.8444793454357549-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", element="0.8444793454357549-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", element="0.8444793454357549-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", element="0.8444793454357549-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", element="0.8444793454357549-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ec484b94c700953d1110e4f8fa87663", el

## Q2. SQL, RDBMS

`products.db` 에 대하여 다음 문제를 해결하세요.

### Q2-1

`products.db`에 존재하는 모든 `table`과 `column`을 조회하세요.

In [7]:
#Q2-1 (190127)

# make connection
con = lite.connect("./products.db")

# get schema from db by pandas
query = "SELECT * FROM sqlite_master;"
schema_series = pd.read_sql(query, con).sql

# check schema
for i, row in schema_series.iteritems():
    print(row)

CREATE TABLE Categories(
  "CategoryID" TEXT,
  "CategoryName" TEXT,
  "Description" TEXT
)
CREATE TABLE Customers(
  "CustomerID" TEXT,
  "CustomerName" TEXT,
  "ContactName" TEXT,
  "Address" TEXT,
  "City" TEXT,
  "PostalCode" TEXT,
  "Country" TEXT
)
CREATE TABLE Employees(
  "EmployeeID" TEXT,
  "LastName" TEXT,
  "FirstName" TEXT,
  "BirthDate" TEXT,
  "Photo" TEXT,
  "Notes" TEXT
)
CREATE TABLE Shippers(
  "ShipperID" TEXT,
  "ShipperName" TEXT,
  "Phone" TEXT
)
CREATE TABLE Suppliers(
  "SupplierID" TEXT,
  "SupplierName" TEXT,
  "ContactName" TEXT,
  "Address" TEXT,
  "City" TEXT,
  "PostalCode" TEXT,
  "Country" TEXT,
  "Phone" TEXT
)
CREATE TABLE Orders(
OrderID integer,
CustomerID integer,
EmployeeID integer,
OrderDate date,
ShipperID integer)
CREATE TABLE OrderDetails(
OrderDetailID integer,
OrderID integer,
ProductID integer,
Quantity integer)
CREATE TABLE Products(
ProductID integer,
ProductName text,
SupplierID integer,
CategoryID integer,
Unit text,
Price integer
)


In [8]:
# select remove target keywords by type
table_remove_keywords = ["CREATE TABLE ", "("]
column_remove_keywords = ["\n", " ", "\"", ",", "TEXT", "integer", "date", "text", ")"]

# split schema string with "\n" and append to schema_list
schema_list = []
for i, row in schema_series.iteritems():
    schema = row.split("\n")
    schema_list.append(schema)

# cleansing data and add to dict
result = {}
for schema in schema_list:
    # assign components to variables
    tbl_name, columns = schema[0], schema[1:]
    # cleansing table_name
    for trk in table_remove_keywords:
        tbl_name = tbl_name.replace(trk, "")
    # cleansing column name
    for crk in column_remove_keywords:
        columns = [column.replace(crk, "") for column in columns if column.replace(crk, "") != '']
    # add to dict
    result[tbl_name] = columns

# print result
for k, v in result.items():
    print(k, v)

Categories ['CategoryID', 'CategoryName', 'Description']
Customers ['CustomerID', 'CustomerName', 'ContactName', 'Address', 'City', 'PostalCode', 'Country']
Employees ['EmployeeID', 'LastName', 'FirstName', 'BirthDate', 'Photo', 'Notes']
Shippers ['ShipperID', 'ShipperName', 'Phone']
Suppliers ['SupplierID', 'SupplierName', 'ContactName', 'Address', 'City', 'PostalCode', 'Country', 'Phone']
Orders ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'ShipperID']
OrderDetails ['OrderDetailID', 'OrderID', 'ProductID', 'Quantity']
Products ['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'Unit', 'Price']


Categories ['CategoryID', 'CategoryName', 'Description']
Customers ['CustomerID', 'CustomerName', 'ContactName', 'Address', 'City', 'PostalCode', 'Country']
Employees ['EmployeeID', 'LastName', 'FirstName', 'BirthDate', 'Photo', 'Notes']
Shippers ['ShipperID', 'ShipperName', 'Phone']
Suppliers ['SupplierID', 'SupplierName', 'ContactName', 'Address', 'City', 'PostalCode', 'Country', 'Phone']
Orders ['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'ShipperID']
OrderDetails ['OrderDetailID', 'OrderID', 'ProductID', 'Quantity']
Products ['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'Unit', 'Price']


### Q2-2

`Customer`의 `Country`가 "Germany" 이거나 `City`가 "London" 인 모든 주문의 주문양(`Quantity`)의 합을 조회하세요.

In [9]:
#Q2-2 (190127)
query = """
SELECT 
    SUM(Quantity) AS TotalQuantity
FROM
    OrderDetails
WHERE 
    OrderID IN
        (SELECT OrderID FROM Orders
        WHERE
            CustomerID IN
                (SELECT CustomerID FROM Customers
                 WHERE Country="Germany" OR City="London"));
"""
pd.read_sql(query, con).iloc[0,0]

2633

In [6]:
#Q2-2

2633

### Q2-3

월별 총 주문 횟수와 총 주문 갯수, 평균 주문 금액(소수점 2째 자리에서 반올림)을 조회하세요.
(총 주문 횟수를 기준으로 내림차순 정렬)

In [10]:
#Q2-3 (190127)
query = """
SELECT 
    SUBSTR(B.OrderDate, 1, 7) AS "MONTH",
    COUNT(A.OrderID) AS "COUNT",
    SUM(A.Quantity) AS "SUM",
    ROUND(SUM(A.Quantity*C.Price) / COUNT(A.OrderID), 2) AS "AVERAGE" 
FROM
    OrderDetails AS A
JOIN
    Orders AS B
    ON A.OrderID = B.OrderID
JOIN
    Products AS C
    ON A.ProductID = C.ProductID
GROUP BY "MONTH"
ORDER BY "COUNT" DESC;
"""
pd.read_sql(query, con)

,MONTH,COUNT,SUM,AVERAGE
0,1997-01,85,2401,981.18
1,1996-12,81,2200,786.68
2,1996-10,73,1738,705.87
3,1996-08,69,1322,482.40
4,1996-11,66,1735,941.88
5,1996-07,59,1462,640.34
6,1996-09,57,1124,606.41
7,1997-02,28,761,713.53


In [7]:
#Q2-3

,MONTH,COUNT,SUM,AVERAGE
0,1997-01,85,2401,981.18
1,1996-12,81,2200,786.68
2,1996-10,73,1738,705.87
3,1996-08,69,1322,482.40
4,1996-11,66,1735,941.88
5,1996-07,59,1462,640.34
6,1996-09,57,1124,606.41
7,1997-02,28,761,713.53


## Q3. noSQL, json, Web Scraping

### Q3-1

다음 금융 시가총액 페이지(http://finance.daum.net/domestic/market_cap) 에서 KOSPI200 상위 30개 종목에 대하여,
뉴스공시 탭의 종목뉴스 최신 100개를 가져와 mLab의 새로운 Collection을 생성하여 저장한 뒤, 모든 뉴스의 `title`, `keywords`, `createdAt` 값을 표현하세요.

`Requirements: requests, json, pymongo`

hint1: 
```python
headers = {"Host":"finance.daum.net"}
requests.get(url, headers=headers)
```

hint2: `some_collection.find(query, {'_id':0,'title':1,})`

<img src="./daum-finance-news.png" width="300">

In [8]:
#Q3-1

{'newsId': 'aStjcgzeqD',
 'title': "[이주의 증시 테마] 게임판호 공급나선 中, 게임주 '훨훨'..반도체 투심 개선",
 'cpKorName': '파이낸셜뉴스',
 'summary': '이번주(21∼25일) 주식시장은 중국 정부가 대형 게임사를 상대로 신규판호(판매허가)를 발급했다는 소식에 게임주의 동반 강세가 나타났다. 또 외국인 순매수세 확대로 삼성전자, SK하이닉스의 주가 회복세가 이어지면서 반도체 관련주가 동반 상승했다. 미세먼지 여파가 계속되는 가운데 인공강우 시험으로 인한 신규 테마주가 생성되기도 했다.',
 'imageUrl': 'https://t1.daumcdn.net/news/201901/26/fnnewsi/20190126110755303nnqp.jpg',
 'keywords': ['게임주', '증시테마', '동반', '인공강우', '중국'],
 'createdAt': '2019-01-26 11:07:55',
 'updatedAt': '2019-01-26 11:07:55'}

{'_id': ObjectId('5c4bd8f6dabf9435efabebca'),
 'code': 'A005930',
 'news': [{'newsId': 'aStjcgzeqD',
   'title': "[이주의 증시 테마] 게임판호 공급나선 中, 게임주 '훨훨'..반도체 투심 개선",
   'cpKorName': '파이낸셜뉴스',
   'summary': '이번주(21∼25일) 주식시장은 중국 정부가 대형 게임사를 상대로 신규판호(판매허가)를 발급했다는 소식에 게임주의 동반 강세가 나타났다. 또 외국인 순매수세 확대로 삼성전자, SK하이닉스의 주가 회복세가 이어지면서 반도체 관련주가 동반 상승했다. 미세먼지 여파가 계속되는 가운데 인공강우 시험으로 인한 신규 테마주가 생성되기도 했다.',
   'imageUrl': 'https://t1.daumcdn.net/news/201901/26/fnnewsi/20190126110755303nnqp.jpg',
   'keywords': ['게임주', '증시테마', '동반', '인공강우', '중국'],
   'createdAt': '2019-01-26 11:07:55',
   'updatedAt': '2019-01-26 11:07:55'},
  {'newsId': 'alLcWoAcoz',
   'title': "4차 산업혁명 핵심동력 5G.. '기회의 땅' 선점 위해 경쟁",
   'cpKorName': '세계일보',
   'summary': '2019년 글로벌 정보통신기술(ICT) 업계의 화두는 단연 ‘5세대(G) 네트워크’다. 우리나라의 경우 지난해 말인 12월 1일 세계 최초 상용화를 시작하면서 다른 나라들보다 출발에서 앞섰다. 해가 바뀌면서 해외에서도 5G에 대한 언급과 기대가 급증한 모습이다. 다만 아직은 구체적으로 눈에 보이는 변화보다는 ‘5G시대의 개막’이라는 슬로건의 의미가 더 큰',
   'imageUrl': 'https://t1.daumcdn.net/news/201901/26/segye/2019012610

### Q3-2
위(mLab Collection)에서 불러온 모든 데이터에 대하여 각 종목별 `keywords`와 그 갯수를 표현하는 `dictionary`를 생성하여 quotes 라는 리스트에 적재하세요.

`Requirements: json, pymongo`

ex)
```python
quotes=[
    {
    'code':'A005930',
    'keywords':{
        '2018년':1,
        '매출액':2,
        ...
        }
    },
...
]
```

In [31]:
#Q3-2

{'code': 'A005930',
 'news': [{'newsId': 'aStjcgzeqD',
   'title': "[이주의 증시 테마] 게임판호 공급나선 中, 게임주 '훨훨'..반도체 투심 개선",
   'cpKorName': '파이낸셜뉴스',
   'summary': '이번주(21∼25일) 주식시장은 중국 정부가 대형 게임사를 상대로 신규판호(판매허가)를 발급했다는 소식에 게임주의 동반 강세가 나타났다. 또 외국인 순매수세 확대로 삼성전자, SK하이닉스의 주가 회복세가 이어지면서 반도체 관련주가 동반 상승했다. 미세먼지 여파가 계속되는 가운데 인공강우 시험으로 인한 신규 테마주가 생성되기도 했다.',
   'imageUrl': 'https://t1.daumcdn.net/news/201901/26/fnnewsi/20190126110755303nnqp.jpg',
   'keywords': ['게임주', '증시테마', '동반', '인공강우', '중국'],
   'createdAt': '2019-01-26 11:07:55',
   'updatedAt': '2019-01-26 11:07:55'},
  {'newsId': 'alLcWoAcoz',
   'title': "4차 산업혁명 핵심동력 5G.. '기회의 땅' 선점 위해 경쟁",
   'cpKorName': '세계일보',
   'summary': '2019년 글로벌 정보통신기술(ICT) 업계의 화두는 단연 ‘5세대(G) 네트워크’다. 우리나라의 경우 지난해 말인 12월 1일 세계 최초 상용화를 시작하면서 다른 나라들보다 출발에서 앞섰다. 해가 바뀌면서 해외에서도 5G에 대한 언급과 기대가 급증한 모습이다. 다만 아직은 구체적으로 눈에 보이는 변화보다는 ‘5G시대의 개막’이라는 슬로건의 의미가 더 큰',
   'imageUrl': 'https://t1.daumcdn.net/news/201901/26/segye/20190126104739000hwwf.jpg',
   'keywords': ['5g', '4차산업혁

## Q3-3
불러온 quotes에 대하여 상위 30개 종목 별 keyword 리스트의 갯수를 구하세요.

hint: 

```python
len_count = {}
for i in map(len, a_words):
	if i in len_count:
		len_count[i] += 1
	else:
		len_count[i] = 1
```

In [29]:
#Q3-3

{'게임주': 1,
 '증시테마': 1,
 '동반': 1,
 '인공강우': 1,
 '중국': 3,
 '5g': 1,
 '4차산업혁명': 1,
 '5g스마트폰': 1,
 '애플': 1,
 '화웨이': 1,
 '스마트폰': 2,
 'mwc': 1,
 '폴더블폰': 2,
 '갤럭시s10': 1,
 '삼성전자': 9,
 'sk하이닉스': 3,
 '상반기': 1,
 '어닝쇼크': 1,
 'd램': 1,
 '전자업계': 1,
 '삼성': 4,
 '아시아경제': 1,
 '엑스레이': 1,
 '후지필름': 1,
 '2017년': 1,
 '매출액': 1,
 '유럽': 1,
 '손': 1,
 'ssd': 1,
 '삼성ssd': 1,
 '게임': 1,
 '카풀': 1,
 '택시업계': 1,
 '택시': 1,
 '신현상': 1,
 '갈등': 1,
 '조선업계': 1,
 '자동차': 1,
 '수주': 1,
 '미국': 3,
 '코스닥': 1,
 '지수': 3,
 '증시': 2,
 '바이코리아': 2,
 '코스피': 2,
 '스마트팩토리': 1,
 '경제적자유': 1,
 '전문가': 1,
 '펀드수익률': 1,
 '수익률': 2,
 '금리': 1,
 '성과': 1,
 '남북경협': 1,
 '주식시장': 1,
 '남북경협주': 1,
 '대기업': 1,
 '에어컨': 1,
 'ai': 1,
 'lg': 1,
 '위니아': 1,
 '다음': 1,
 '적자': 1,
 '환골탈태': 1,
 '빅딜': 1,
 '하이닉스': 1,
 '중견련': 1,
 '사내벤처': 1,
 '연임': 1,
 'ces2019': 1,
 '종가': 1,
 '무역협상': 1,
 '삼성sdi': 1,
 '기업공시': 1,
 '인수': 1,
 '삼성중공업': 1}